In [1]:
using TSSOS
using DynamicPolynomials
using CSV
using DataFrames
using Statistics

In [3]:
dis = CSV.read("distanceSet.csv", header=1);
taxi=[1.38,0.77,2.52,1.15,1.2,3.53,1.0,2.2,0.95,1.42,1.7,2.77,0.57, 0.81,21.47, 1.13,1.19, 0.83, 0.93, 3.3,0.71, 1.04, 11.2, 0.72, 2.63, 0.82, 10.32, 3.51,7.25, 0.48, 2.5, 1.5, 0.4, 1.86, 2.34, 1.42,2.6,1.4, 1.02, 5.96, 0.84, 1.2, 9.3, 1.66,2.0, 2.55, 2.04, 17.87,0.6, 0.57];

In [32]:
# new formulation 3 19/11
# calculate 3 objective: intra-fair inter-fair customer-care

function F3_tssos(icount,jcount,taxi,lam1,lam2,lam3)
    #icount=5 # the number of customers
    #jcount=5 # the number of drivers
    #female=[2,4]
    #male=[1,3,5]
    female=collect(2:2:jcount)
    male=collect(1:2:jcount)
    
    mu1=10
    mu2=10
    mu3=0

    #lam1=1
    #lam2=1
    #lam3=0
    
    dis=rand(icount,jcount)
    
    # set nc operators
    @polyvar u[1:jcount] M[1:icount*jcount]; #z[1:jcount]; 
    var=vcat(u,M);

    #l2=[z[j]-1+(u[j]+0.01)/1.01 for j in 1:jcount];
    #l3=[z[j]+1-(u[j]+0.01)/1.01 for j in 1:jcount];
    #l4=[b1-(1/length(female))*sum(z[j] for j in female),b1-(1/length(male))*sum(z[j] for j in male)];
    #l5=[b2-sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount) for i in 1:icount];
    l6=[1-sum(M[(i-1)*(jcount)+j] for i in 1:icount) for j in 1:jcount];
    l7=[M[(i-1)*(jcount)+j]*(M[(i-1)*(jcount)+j]-1) for i in 1:icount for j in 1:jcount]; #equality #numeq=25

    obj=lam1*sum((1-(u[j]+0.01)/1.01)^2 for j in 1:jcount)+lam2*(sum((u[j]+0.01)/1.01 for j in female)-sum((u[j]+0.01)/1.01 for j in male))^2+lam3*sum(M[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)+sum(mu1*(sum(M[(i-1)*(jcount)+j]*(taxi[i]-dis[i,j]) for i in 1:icount)-u[j])^2  for j in 1:jcount)+sum(mu2*(sum(M[(i-1)*(jcount)+j] for j in 1:jcount)-1)^2  for i in 1:icount)

    # pop
    pop=vcat(obj,l6,l7);

    d=1; # the relaxation order
    opt,sol,data=cs_tssos_first(pop,var,d,numeq=icount*jcount,TS="MD",solution=true);
    #opt,sol,data=cs_tssos_higher!(data,TS="MD")
    
    u_sol=sol[1:jcount]
    M_sol=sol[jcount+1:end]
    #intra5=sum((1-(u_sol[j]+0.01)/1.01)^2 for j in 1:jcount)
    
    mu=sum((u_sol[j]+0.01)/1.01 for j in 1:jcount)/jcount
    muf=sum((u_sol[j]+0.01)/1.01 for j in female)/length(female)
    mum=sum((u_sol[j]+0.01)/1.01 for j in male)/length(male)
    
    intra4=1/(2*mu*jcount^2)*sum(abs((u_sol[k]+0.01)/1.01-(u_sol[j]+0.01)/1.01) for k in 1:jcount for j in 1:jcount)
    inter3=(sum((u_sol[j]+0.01)/1.01 for j in female)-sum((u_sol[j]+0.01)/1.01 for j in male))^2
    care=sum(M_sol[(i-1)*(jcount)+j]*dis[i,j] for j in 1:jcount for i in 1:icount)
    
    intra2=abs(1/jcount*sum(((u_sol[j]+0.01)/(1.01*mu))*log((u_sol[j]+0.01)/(1.01*mu)) for j in 1:jcount))
    intra3=abs(1/jcount*sum(log((1.01*mu)/(u_sol[j]+0.01)) for j in 1:jcount))
    
    inter1=abs((length(female)/jcount)*muf/mu*log(muf/mu)+(length(male)/jcount)*mum/mu*log(mum/mu))
    inter2=abs((length(female)/jcount)*log(mu/muf)+(length(male)/jcount)*log(mu/mum))
    
    return [intra2,intra3,intra4,inter1,inter2,inter3,care]
    #return [intra4,inter3,care]

end

F3_tssos (generic function with 1 method)

In [33]:
F3_tssos(5,5,taxi[1:5],0.5,0.5,0)

------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve te

7-element Array{Float64,1}:
 0.014244166433593611
 0.013803539392723739
 0.08636167013304498
 0.012512908004531934
 0.012351782298364561
 0.04063770364833207
 2.5180908064486256

In [34]:
# for pareto optimal
icount=5 # the number of customers
jcount=5 # the number of drivers
repeat=10
lam1_range=0:0.1:1  
#lam2_range=1:-0.1:0 #step 0.1
df = DataFrame(Intra_Fair2=Float64[],Intra_Fair3=Float64[],Intra_Fair4=Float64[],Inter_Fair1=Float64[],Inter_Fair2=Float64[],Inter_Fair3=Float64[],Customer_Care=Float64[],lam1=Float64[],lam2=Float64[],lam3=Float64[])

for lam1 in lam1_range
    for lam2 in 0:0.1:(1-lam1)
        for r in 1:repeat
            obj=F3_tssos(icount,jcount,taxi[1:5],lam1,lam2,1-lam1-lam2)
            push!(df, push!(obj,lam1,lam2,1-lam1-lam2))
        end
    end
end

#df[!,:lam2] .= -df[!,:lam1].+1
#df[!,:lam3] .= 0

#CSV.write("ParetoOptimal.csv", df,header=true)

------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve term

5   2.3e+00  4.0e-02  2.6e-02  2.20e+00   -4.804706973e+01  -4.835370686e+01  4.0e-02  0.11  
6   4.2e-01  7.3e-03  1.6e-03  1.59e+00   -4.905767967e+00  -4.938222758e+00  7.3e-03  0.11  
7   8.4e-02  1.5e-03  1.4e-04  1.09e+00   4.507790247e-01   4.452649178e-01   1.5e-03  0.13  
8   2.6e-02  4.4e-04  2.1e-05  1.00e+00   8.208814886e-01   8.195767808e-01   4.4e-04  0.14  
9   6.8e-03  1.2e-04  2.8e-06  1.01e+00   9.752981361e-01   9.750056740e-01   1.2e-04  0.16  
10  1.3e-03  2.2e-05  2.2e-07  1.00e+00   9.940239078e-01   9.939748042e-01   2.2e-05  0.16  
11  4.5e-04  7.8e-06  4.4e-08  1.00e+00   1.000316949e+00   1.000300889e+00   7.8e-06  0.17  
12  1.7e-04  3.0e-06  1.1e-08  9.99e-01   1.002386046e+00   1.002379881e+00   3.0e-06  0.19  
13  3.2e-05  5.6e-07  8.3e-10  9.99e-01   1.003594402e+00   1.003593405e+00   5.6e-07  0.20  
14  5.5e-06  9.4e-08  5.9e-11  1.00e+00   1.003967076e+00   1.003966891e+00   9.4e-08  0.20  
15  4.2e-07  7.4e-09  1.3e-12  1.00e+00   1.004035021e+00   

optimum = 1.7098866990150885
------------------------------------------------------
The clique sizes of varibles:
[16, 14, 9, 6]
[1, 2, 5, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 296             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 57              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0

Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 2.17e+04          after factor           : 2.67e+04        
Factor     - dense dim.             : 0                 flops                  : 3.85e+06        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   5.8e+01  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  0.03  
1   4.0e+01  6.9e-01  8.1e-01  -8.83e-01  -2.782754674e+00  -3.183702811e+00  6.9e-01  0.11  
2   3.2e+01  5.5e-01  7.1e-01  -9.27e-01  -4.157971232e+01  -4.229409429e+01  5.5e-01  0.14  
3   1.1e+01  1.9e-01  3.8e-01  -8.74e-01  -2.091878371e+02  -2.123521600e+02  1.9e-01  0.16  
4   4.3e+00  7.4e-02  9.0e-02  -3.35e-01  -2.102236033e+02  -2.113504327e+02  7.4e-02  0.19  
5   2.3e+00  3.9e-02  2.5e-02  2.19e+00   -4.613886701e+01  -4.642140344e+01  3.9e-02  0.22  
6   4.0e-01  6.8e-03  1.4e-03  1.58e+00   -4.410

4   5.0e+00  7.5e-02  9.1e-02  -3.34e-01  -2.063573200e+02  -2.074449827e+02  7.5e-02  0.13  
5   2.6e+00  3.9e-02  2.4e-02  2.16e+00   -4.511445081e+01  -4.537774893e+01  3.9e-02  0.14  
6   5.4e-01  8.1e-03  1.8e-03  1.58e+00   -5.437288443e+00  -5.468408894e+00  8.1e-03  0.16  
7   1.4e-01  2.1e-03  2.2e-04  1.10e+00   -3.790691331e-01  -3.859665139e-01  2.1e-03  0.19  
8   3.9e-02  5.8e-04  3.1e-05  1.03e+00   1.995359698e-01   1.979828526e-01   5.8e-04  0.20  
9   6.8e-03  1.0e-04  2.1e-06  1.02e+00   4.374581294e-01   4.372385435e-01   1.0e-04  0.22  
10  1.7e-03  2.6e-05  2.6e-07  9.93e-01   4.416922270e-01   4.416403546e-01   2.6e-05  0.25  
11  3.8e-04  5.8e-06  2.7e-08  9.99e-01   4.463560260e-01   4.463456388e-01   5.8e-06  0.27  
12  1.4e-04  2.1e-06  5.8e-09  9.99e-01   4.479603551e-01   4.479566524e-01   2.1e-06  0.28  
13  4.1e-05  6.2e-07  9.6e-10  1.00e+00   4.483789628e-01   4.483778214e-01   6.2e-07  0.30  
14  1.0e-05  1.5e-07  1.2e-10  1.00e+00   4.485448398e-01   

...

------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max        

4   3.9e+00  5.7e-02  3.8e-02  2.77e-03   -1.100970308e+02  -1.103040283e+02  5.7e-02  0.08  
5   1.0e+00  1.5e-02  3.9e-03  1.84e+00   -1.459888002e+01  -1.462793116e+01  1.5e-02  0.09  
6   2.0e-01  2.9e-03  3.2e-04  1.19e+00   -1.380775987e+00  -1.386190807e+00  2.9e-03  0.13  
7   4.6e-02  6.8e-04  3.7e-05  1.05e+00   7.371937948e-01   7.357851022e-01   6.8e-04  0.14  
8   1.4e-02  2.1e-04  6.4e-06  1.02e+00   9.070545919e-01   9.066116509e-01   2.1e-04  0.16  
9   3.7e-03  5.4e-05  8.3e-07  1.01e+00   9.847871247e-01   9.846698859e-01   5.4e-05  0.19  
10  8.7e-04  1.3e-05  9.4e-08  1.00e+00   1.000308827e+00   1.000281972e+00   1.3e-05  0.20  
11  1.7e-04  2.5e-06  8.2e-09  9.99e-01   1.005030577e+00   1.005025162e+00   2.5e-06  0.23  
12  3.7e-05  5.4e-07  8.2e-10  1.00e+00   1.005702597e+00   1.005701413e+00   5.4e-07  0.25  
13  3.6e-06  5.2e-08  2.4e-11  1.00e+00   1.005862055e+00   1.005861948e+00   5.2e-08  0.25  
14  1.8e-07  3.0e-09  2.8e-13  1.00e+00   1.005871071e+00   

optimum = 0.8974942002198315
------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               
  Scalar variables       : 31              
  Matrix variables       : 52              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 :

Excessive output truncated after 524288 bytes.


------------------------------------------------------
The clique sizes of varibles:
[20, 16, 13, 10]
[1, 2, 1, 5]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 366             
  Cones                  : 0               


In [35]:
CSV.write("ParetoOptimal_1.csv", df,header=true)

"ParetoOptimal_1.csv"